In [3]:
import pandas as pd
import numpy as np

np.random.seed(123)

In [5]:
df = pd.read_csv('students.csv', index_col=0)

## Reshaping

We will talk about reshaping operations in more detail when we discuss tidy data, but for now we will focus on a couple of common operations that can be used to summarize our data by different subgroups.

### `pd.crosstab`

For an example of `.crosstab`, we will count the number of students passing math in each classroom.

In [7]:
#  cant do this : df.crosstab(df.passing_math, df.classroom)

In [8]:
df.head()

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A
3,Billy,98,96,88,B
4,Ada,77,92,98,A


In [9]:
df['passing_math'] = np.where(df.math >=70, 'Pass', 'Fail')

In [10]:
df.head()

,name,math,english,reading,classroom,passing_math
0,Sally,62,85,80,A,Fail
1,Jane,88,79,67,B,Pass
2,Suzie,94,74,95,A,Pass
3,Billy,98,96,88,B,Pass
4,Ada,77,92,98,A,Pass


In [11]:
df.passing_math.value_counts()

Pass    10
Fail     2
Name: passing_math, dtype: int64

In [13]:
# We will use our student grades DataFrame, df.
pd.crosstab(df.classroom, df.passing_math)


passing_math,Fail,Pass
classroom,,
A,2,6
B,0,4


We'll use the `pd.crosstab` function to count the number of occurances of each subgroup (i.e. each unique combination of classroom and whether or not the student is passing math):

We can also view subtotals with the `margins` set to `True`.

In [14]:
pd.crosstab(df.classroom, df.passing_math, margins=True)

passing_math,Fail,Pass,All
classroom,,,
A,2,6,8
B,0,4,4
All,2,10,12


The `.crosstab` function will let us view the numbers as percentages of the total as well by setting `normalize` to `True`.

In [15]:
pd.crosstab(df.classroom, df.passing_math, margins=True, normalize=True)

passing_math,Fail,Pass,All
classroom,,,
A,0.166667,0.500000,0.666667
B,0.000000,0.333333,0.333333
All,0.166667,0.833333,1.000000


### `.pivot_table`

Here we use the `.pivot_table` method to create our summary. This method produces output similar to an excel pivot table. We must supply 3 things here:

- which values will make up the rows (the `index`)
- which values will make up the columns
- the values we are aggregating
- an aggregation method (`aggfunc`); if we can omit this, and `mean` will be used by default

For an example using the `pivot_table` method, we'll calculate the average math grade for the combination of `classroom` and `passing_math` status.

In [17]:
df.pivot_table(index='classroom', columns='passing_math', values='math', aggfunc='max')

passing_math,Fail,Pass
classroom,,
A,69.0,94.0
B,NaN,98.0


Here we'll create a dataframe that represents various orders at a restaurant.

In [18]:
n = 40

orders = pd.DataFrame({
    'drink': np.random.choice(['Tea', 'Water', 'Water'], n),
    'meal': np.random.choice(['Curry', 'Yakisoba Noodle', 'Pad Thai'], n),
})

orders.sample(10)

,drink,meal
38,Water,Pad Thai
29,Water,Yakisoba Noodle
37,Tea,Curry
16,Tea,Curry
33,Tea,Pad Thai
31,Tea,Yakisoba Noodle
8,Water,Yakisoba Noodle
5,Water,Curry
0,Water,Yakisoba Noodle
15,Water,Pad Thai


#### `.map`

The `.map` method lets us use a dictionary to calculate the total price for an order; then I can save my calculations to a new column named `bill`. Let's do this step-by-step.

In [19]:
# Create a dictionary of prices for drinks and meals.

prices = {
    'Yakisoba Noodle': 9,
    'Curry': 11,
    'Pad Thai': 10,
    'Tea': 2,
    'Water': 0,
}

In [20]:
"""
Match the values in the 'drink' and 'meal' columns with the values in the 'prices' dictionary 
and perform the specified calculation. Save this calculation to a new column named 'bill'.
"""

orders['bill'] = orders.drink.map(prices) + orders.meal.map(prices)

In [22]:
orders.head()

,drink,meal,bill
0,Water,Yakisoba Noodle,9
1,Water,Pad Thai,10
2,Water,Pad Thai,10
3,Water,Pad Thai,10
4,Tea,Yakisoba Noodle,11


Let's take a look at how many orders have each combination of meal and drink:

In [23]:
pd.crosstab(orders.drink, orders.meal)

meal,Curry,Pad Thai,Yakisoba Noodle
drink,,,
Tea,5,6,4
Water,6,12,7


And let's find out the average bill amount for each combination: 

In [25]:
orders.pivot_table(index='drink', columns='meal', values='bill')

meal,Curry,Pad Thai,Yakisoba Noodle
drink,,,
Tea,13,12,11
Water,11,10,9


It's interesting to note that we could find the same information with a multi-level group by:

In [26]:
orders.groupby(['meal','drink']).bill.agg("mean")

meal             drink
Curry            Tea      13.0
                 Water    11.0
Pad Thai         Tea      12.0
                 Water    10.0
Yakisoba Noodle  Tea      11.0
                 Water     9.0
Name: bill, dtype: float64

The choice between group by and a pivot table here is mostly asthetic, and you should use whichever makes more sense to you with the problem at hand. 

### Transposing

In [28]:
orders.pivot_table(index='drink', columns='meal', values='bill').T

drink,Tea,Water
meal,,
Curry,13,11
Pad Thai,12,10
Yakisoba Noodle,11,9


In [29]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
math,12.0,84.833333,11.134168,62.0,78.50,90.0,92.25,98.0
english,12.0,77.666667,13.371158,62.0,63.75,77.5,86.75,99.0
reading,12.0,86.500000,9.643651,67.0,80.75,89.0,93.25,98.0


## Exercises III

1. Use your `get_db_url` function to help you explore the data from the `chipotle` database. 

2. What is the total price for each order?

3. What are the most popular 3 items?

4. Which item has produced the most revenue?

5. Using the `titles` DataFrame, visualize the number of employees with each title.

6. Join the `employees` and `titles` DataFrames together.

7. Visualize how frequently employees change titles.

8. For each title, find the hire date of the employee that was hired most recently with that title.

9. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)